# Pre-Post Analysis Of Opioid Overdose Deaths In Florida

In [1]:
# importing all the libraries that maybe required 

import pandas as pd
import numpy as np
import altair as alt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# loading data on deaths in Florida and its comparison states - Pennsylvania (PA), Michigan (MI), and North Carolina (NC)
deaths = pd.read_csv(r"C:\Users\annap\Downloads\deaths_fl.csv")

In [3]:
deaths.head(3)

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State,StateFIPS,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,state_abbrev,FIP_unique,Population,county_test
0,Alachua,12001.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,17.0,FL,12,Alachua,Florida,12001,FL,FL | ALACHUA,FL,1200112,242685.0,Alachua
1,Alachua,12001.0,2007.0,2007.0,All other alcohol-induced causes,A9,22.0,FL,12,Alachua,Florida,12001,FL,FL | ALACHUA,FL,1200112,242685.0,Alachua
2,Alachua,12001.0,2007.0,2007.0,All other non-drug and non-alcohol causes,O9,1578.0,FL,12,Alachua,Florida,12001,FL,FL | ALACHUA,FL,1200112,242685.0,Alachua


In [4]:
# creating a copy of deaths dataset, to prevent SettingWithCopy warnings. The irrelevant columns are dropped.

deaths_copy = deaths.copy()
deaths_copy.drop(
    [
        "County Code",
        "Year Code",
        "Drug/Alcohol Induced Cause Code",
        "State",
        "StateFIPS",
        "CountyName",
        "CountyFIPS",
        "StateAbbr",
        "STATE_COUNTY",
        "state_abbrev",
        "FIP_unique",
        "county_test",
    ],
    axis=1,
    inplace=True,
)

In [5]:
# checking the data after dropping the irrelevant columns
deaths_copy.head(3)

,County,Year,Drug/Alcohol Induced Cause,Deaths,StateName,Population
0,Alachua,2007.0,Drug poisonings (overdose) Unintentional (X40-...,17.0,Florida,242685.0
1,Alachua,2007.0,All other alcohol-induced causes,22.0,Florida,242685.0
2,Alachua,2007.0,All other non-drug and non-alcohol causes,1578.0,Florida,242685.0


In [6]:
# converting some of the columns to more appropriate data type - integer data type
deaths_copy["Year"] = deaths_copy["Year"].astype("int64")
deaths_copy["Deaths"] = deaths_copy["Deaths"].astype("int64")
deaths_copy["Population"] = deaths_copy["Population"].astype("int64")


# renaming the column "StateName" for simplicity
deaths_copy.rename(
    columns={
        "StateName": "State",
    },
    inplace=True,
)


In [7]:
# checking the data after making the modifications
deaths_copy.head(3)

,County,Year,Drug/Alcohol Induced Cause,Deaths,State,Population
0,Alachua,2007,Drug poisonings (overdose) Unintentional (X40-...,17,Florida,242685
1,Alachua,2007,All other alcohol-induced causes,22,Florida,242685
2,Alachua,2007,All other non-drug and non-alcohol causes,1578,Florida,242685


In [8]:
# checking to see all the deaths caused due to drugs or alcohol 
deaths_copy["Drug/Alcohol Induced Cause"].unique()

array(['Drug poisonings (overdose) Unintentional (X40-X44)',
       'All other alcohol-induced causes',
       'All other non-drug and non-alcohol causes',
       'Drug poisonings (overdose) Suicide (X60-X64)',
       'All other drug-induced causes',
       'Drug poisonings (overdose) Undetermined (Y10-Y14)',
       'Alcohol poisonings (overdose) (X45, X65, Y15)'], dtype=object)

In [9]:
# creating the dataset for Florida which has all deaths that are drug overdose related 

florida_deaths = deaths_copy[
    (deaths_copy["State"] == "Florida")
    & (
        (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Unintentional (X40-X44)"
        )
        | (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Undetermined (Y10-Y14)"
        )
        | (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Suicide (X60-X64)"
        )
    )
]

In [10]:
florida_deaths.head(5)


,County,Year,Drug/Alcohol Induced Cause,Deaths,State,Population
0,Alachua,2007,Drug poisonings (overdose) Unintentional (X40-...,17,Florida,242685
5,Alachua,2009,Drug poisonings (overdose) Unintentional (X40-...,15,Florida,246657
8,Alachua,2010,Drug poisonings (overdose) Unintentional (X40-...,15,Florida,247614
11,Alachua,2011,Drug poisonings (overdose) Unintentional (X40-...,14,Florida,249834
14,Alachua,2012,Drug poisonings (overdose) Unintentional (X40-...,14,Florida,251520


In [11]:
florida_deaths_copy = florida_deaths.copy()

In [12]:
# calculating the drug related death rates in Florida per 100000 people

florida_deaths_copy["Opioid_Overdose_Deaths_Per_100K"] = (
    florida_deaths_copy["Deaths"] / florida_deaths_copy["Population"]
) * 100_000

florida_deaths_copy.head(3)


,County,Year,Drug/Alcohol Induced Cause,Deaths,State,Population,Opioid_Overdose_Deaths_Per_100K
0,Alachua,2007,Drug poisonings (overdose) Unintentional (X40-...,17,Florida,242685,7.004965
5,Alachua,2009,Drug poisonings (overdose) Unintentional (X40-...,15,Florida,246657,6.081319
8,Alachua,2010,Drug poisonings (overdose) Unintentional (X40-...,15,Florida,247614,6.057816


In [13]:
# calculating and displaying year and county wise results for drug related deaths in Florida

florida_result= (
    florida_deaths_copy.groupby(["Year", "County"])["Opioid_Overdose_Deaths_Per_100K"]
    .sum()
    .reset_index()
)
florida_result.head(5)

,Year,County,Opioid_Overdose_Deaths_Per_100K
0,2007,Alachua,7.004965
1,2007,Bay,18.143881
2,2007,Brevard,19.269286
3,2007,Broward,15.225255
4,2007,Citrus,12.058961


In [14]:
# calculating and displaying yearwise results for drug related deaths in Florida

florida_result_yearwise = florida_result.groupby("Year")[
    "Opioid_Overdose_Deaths_Per_100K"
].mean()

florida_result_yearwise

Year
2007    15.128254
2008    16.414020
2009    16.082433
2010    16.766656
2011    15.933404
2012    14.203665
2013    12.772121
Name: Opioid_Overdose_Deaths_Per_100K, dtype: float64

In [15]:
 # splitting data into two subsets - before policy change in 2010 and (or in)after policy change in 2010

florida_before = florida_result[florida_result["Year"] < 2010]
florida_after = florida_result[florida_result["Year"] >= 2010]
 


In [16]:
# checking data in florida subset before policy change in 2010 
florida_before.head(3)

,Year,County,Opioid_Overdose_Deaths_Per_100K
0,2007,Alachua,7.004965
1,2007,Bay,18.143881
2,2007,Brevard,19.269286


In [17]:
# checking data in florida subset after policy change in 2010
florida_after.head(3)

,Year,County,Opioid_Overdose_Deaths_Per_100K
101,2010,Alachua,6.057816
102,2010,Bay,13.001903
103,2010,Brevard,24.817773


In [18]:
# Function to plot a vertical line at year of policy implementation
def vertical_line(year):
    
    line = alt.Chart(pd.DataFrame({
    'Year': [year],
    'color': ["black"]
    })).mark_rule().encode(
    x='Year:Q',
    color=alt.Color('color:N', scale=None)
    )

    return line

In [19]:
# creating the vertical line at 2010, the year of implementation of policy changes in Florida
line = vertical_line(2010)

In [20]:
# creating the regression model and calculating the error bands for creating the plots 

def get_reg_fit_and_ci(data, color, xvar, yvar, legend, alpha=0.05):
    
    # Creating the grid for predicted values 
    colour = color
    years = [2007,2008,2009,2010,2011,2012,2013] 
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fitting the model and making the predictions 
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)
    
    # Utilizing the predictions to create the points and error bands in the chart 
    predictions["Before/After"] = f"{legend}"
    reg = (
        alt.Chart(predictions)
        .mark_line()
        .encode(
            x=xvar,
            y=alt.Y(yvar),
            color = alt.Color("Before/After",legend=alt.Legend(title = "Before/After Policy Change"))
        )
    )

    ci = (
        alt.Chart(predictions)
        .mark_errorband()
        .encode(
            alt.X(f"{xvar}:Q", axis=alt.Axis(format=".0f", values=years)),
            y=alt.Y(
                "ci_low",
                title="Opioid Overdose Deaths Per 100K Residents",
                scale=alt.Scale(zero=False),
            ),
            y2="ci_high",
            color=alt.value(f"{color}"),
        )
    )
    chart = ci + reg
    return predictions, chart



In [21]:
# Using the get_reg_fit_and_ci function to create the charts 

def build_chart(data, color, xvar, yvar, legend, alpha=0.05):
    fit, reg_chart = get_reg_fit_and_ci(
        data=data, color=color, xvar=xvar, yvar=yvar,  legend=legend, alpha=alpha,
    )
    return reg_chart



In [22]:
# creating the final plot for pre-post analysis of drug overdose deaths in Florida 

florida_before_chart = build_chart(
    florida_before, "orange", "Year","Opioid_Overdose_Deaths_Per_100K", "Florida Before",  alpha=0.05
)

florida_after_chart = build_chart(
    florida_after, "blue", "Year", "Opioid_Overdose_Deaths_Per_100K", "Florida After", alpha=0.05
)

florida_final_pre_post = (florida_before_chart + florida_after_chart + line).properties(title="Pre Post Analysis Of Opioid Overdose Deaths In Florida")

florida_final_pre_post


alt.LayerChart(...)